

Dự đoán độ tương quan giữa từ khóa tìm kiếm và sản phẩm trả về với dữ liệu từ HomeDepot.com
--------------------------------------------------------


Đây chỉ là là cell được cung cấp bởi Kaggle.  
  
Nó import thư viện **numpy** cho việc tính toán toán học, **pandas** cho việc xử lý dữ liệu, **matplotlib** cho việc vẽ biểu đồ, và **re** cho việc xử lý ký pháp **regex**.  
  
Ngoài ra, kaggle còn cho ta xem đường dẫn đến dữ liệu với hàm **ls** trong linux để liệt kê các thư mục / files trong đường dẫn mà họ cho trước.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import re

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

**Tổng quan**
--------------
  
Home Depot đã xây dựng ra bộ dữ liệu được người đánh giá về mức độ liên quan giữa kết quả tìm kiếm và từ tìm kiếm, có khả năng thay đổi tích cực đến chất lượng dịch vụ.  
  
Notebook này sẽ xây dựng một mô hình đánh giá độ tương quan giữa các từ khóa tìm kiếm và kết quả trả về, dựa trên dữ liệu của Home Depot.  

# Mục lục  
* [Đọc dữ liệu](#read_data)  
* [Khám phá dữ liệu](#data_exploration)  
    - [Khám phá dữ liệu training](#training_data_exploration)
    - [Khám phá dữ liệu testing](#testing_data_exploration)
    - [Khám phá dữ liệu attribute](#attribute_data_exploration)
    - [Khám phá dữ liệu description](#description_data_exploration)
* [Xử lý, phân tích, và xây dựng dữ liệu](#data_processing)  
    - [Xây dựng dữ liệu brand mới, join brand và description](#new_brand)
    - [Xây dựng dữ liệu attribute mới](#new_attribute)
    - [Tạo ra một số dữ liệu định lượng](#new_quantitative)
    - [Tiền xử lý](#pre_processing)
        - [Chuẩn hóa các xâu](#normalization)
        - [Stemming](#stem)
* [Biểu diễn dữ liệu](#data_representation)  
    - [Tf-idf](#tfidf)
    - [TruncatedSVD](#svd)
    - [Pipeline dữ liệu](#tfidf_svd)
    - [Hàm mất mát](#loss)
    - [Phân tách dữ liệu train-test](#data_split)
* [Huấn luyện mô hình](#model_training)
    - [Khởi tạo mô hình](#model_selection)
    - [Tìm kiếm bộ tham số tối ưu](#find_best_param)
    - [Huấn luyện với bộ tham số tối ưu](#best_param_training)
    - [Dự đoán kết quả cuối cùng](#prediction)
* [Kết quả ](#result)

<a id="read_data"></a>
# Đọc dữ liệu  

Giờ ta sẽ cần đọc dữ liệu csv từ nguồn dữ liệu được lưu trữ dưới dạng **.csv.zip**.  

Sử dụng hàm **pandas.read_csv**, ta sẽ có khả năng giải nén đồng thời trong quá trình đọc. 

Các file **.zip** tự động được giải nén và đọc dưới dạng **pd.DataFrame**.  
Các dữ liệu được đọc là "**training_data**", "**testing_data**", "**attribute_data**", và "**description**".  
  
Dựa vào tên ta có thể đoán được các dữ liệu này làm gì, tuy nhiên để chắc chắn ta sẽ cần khám phá từng cái một.  

In [ ]:
# Load files
training_data = pd.read_csv("../input/home-depot-product-search-relevance/train.csv.zip", encoding="ISO-8859-1")
testing_data = pd.read_csv("../input/home-depot-product-search-relevance/test.csv.zip", encoding="ISO-8859-1")
attribute_data = pd.read_csv('../input/home-depot-product-search-relevance/attributes.csv.zip')
descriptions = pd.read_csv('../input/home-depot-product-search-relevance/product_descriptions.csv.zip')

<a id="data_exploration"></a>
# Khám phá dữ liệu

<a id="training_data_exploration"></a>
## Khám phá training_data

Hiện ta không biết **training_data** mang ý nghĩa gì.  
  
Tiếp cận đầu tiên là tên các cột của các dữ liệu. Ta sẽ sử dụng hàm **pandas.DataFrame.columns.values** trả về một list các tên của các cột trong DataFrame đó.  
  
**Training_data** có 5 cột, dựa vào tên ta có thể suy ra được:  
- **product_uid**: Mã của sản phẩm. Các sản phẩm khác nhau sẽ có mã sản phẩm khác nhau.  
- **product_title**: Tiêu đề tên sản phẩm.  
- **search_term**: Từ khóa tìm kiếm.  
- **relevance**: Điểm tương quan như đã mô tả.  
  
Như vậy, đây là dữ liệu huấn luyện của ta.    

In [ ]:
print("Train data columns are: {}".format(training_data.columns.values))
print("Test data columns are: {}".format(testing_data.columns.values))
print("Attribute data columns are: {}".format(attribute_data.columns.values))
print("Description data columns are: {}".format(descriptions.columns.values))

Giờ đã biết ý nghĩa của các trường thông tin, ta muốn hiểu rõ hơn về kiểu dữ liệu trong dataframe này.  
  
Hàm **pandas.DataFrame.info()** sẽ cho ta biết các cột mang những giá trị như thế nào.  
  
Nhìn vào kết quả, ta thấy dữ liệu có **74067**, hoàn toàn không có dữ liệu **null**.  
Việc này khá tuyệt, ta sẽ không phải xử lý dữ liệu **null** trong bảng.  
  
Tuy nhiên, **product_title** và **search_term** mang loại object, ta sẽ cần xem chi tiết hơn về chúng.  

In [ ]:
training_data.info()

Ta muốn xem một số hàng đầu của **product_title** và **search_term**.  
Hàm **pandas.DataFrame.head()** trả về 5 hàng đầu của dữ liệu.  
  
Ta nhận thấy Tiêu đề này chứa các dữ liệu văn bản chưa được chuẩn hóa, i.e chứa các ký tự in hoa lẫn thường, ký tự đặc biệt, ... Ta sẽ cần chuẩn hóa lại để tiện cho việc xử lý sau này.  

In [ ]:
training_data[['product_title', 'search_term']].head()

Ta cũng muốn tìm hiểu dữ liệu **relevance** được phân bố thế nào.  
  
Hàm **pandas.DataFrame.describe()** sẽ cho chúng ta dữ liệu thống kê về nó.  
  
Phần lớn độ tương quan đánh giá sẽ nằm trong đoạn 2.00 đến 3.00 (50% percentile là 2.33).  
Dữ liệu sẽ chênh nhau khá lớn về các mẫu có độ tương quan lớn.

In [ ]:
training_data['relevance'].describe()

Để nhìn rõ hơn sự phân bố của **relevance**, ta sử dụng hàm **pandas.DataFrame.Series.hist()** để vẽ hiểu đồ histogram.  
  
Kết quả ta nhận thấy cũng giống như đã xem ở số liệu thống kê, với số lượng mẫu mang độ relevance chiếm đại đa số.

In [ ]:
training_data.relevance.hist()

<a id="testing_data_exploration"></a>
Lúc này, ta đoán dữ liệu **testing_data** sẽ mang tính kiểm duyệt để nộp ra kết quả cuối cùng, do đó nó cũng mang dữ liệu tương tự với **training_data**, chỉ bỏ đi trường **relevance**.  
  
Ta kiểm tra bằng hàm **pandas.DataFrame.info()** và kết quả đúng như vậy.  

In [ ]:
testing_data.info()

<a id="attribute_data_exploration"></a>
## Khám phá dữ liệu thông tin thuộc tính

Ta muốn xem **attribute_data** mang những thông tin gì.  
  
Ta dùng hàm **pandas.DataFrame.head()** để xem một số dòng đầu của dữ liệu.  
  
Dựa vào kết quả thu được, ta có một số nhận xét sau:  
- **product_uid**: Mã sản phẩm. Mã này lại được ghi dưới dạng số thập phân, ta sẽ cần sửa lại thành dạng số nguyên để đồng nhất với các dữ liệu khác. <a id="float-id"></a> 
- **name**: Tên thuộc tính. Mỗi sản phẩm theo product_uid có thể mang nhiều thuộc tính. Tên các thuộc tính đã được giấu đi và thay thế bằng các cụm từ gồm chữ latin in hoa, thường và số.  
- **value**: Thông tin về thuộc tính tương ứng.  Trường thông tin này mang nhiều loại ký tự.  
  
Nhìn vào kết quả, ta sẽ cần xem xét thêm trường **name**, khi thấy nó mang những giá trị "**bullet**" (các chấm đầu dòng thay vì một từ mang ý nghĩa nào đó).

In [ ]:
print(attribute_data.head())

Như đã nhắc đến ở cell bên trên, ta sẽ thử xem trường **name** mang các dữ liệu thế nào.  
  
Nhìn vài kết quả, ta thấy nhiều tên thuộc tính là **Bullet**, tức là các gạch đầu dòng, không mang ý nghĩa gì. Tuy nhiên, lại có những thông tin mang ngữ nghĩa.  
  
Có 5411 trường thông tin khác nhau, trên tổng số 2 triệu mẫu, tức là ta hoàn toàn có khả năng dựa vào thông tin này để lấy đặc trưng.  

In [ ]:
print(attribute_data['name'].unique())
print(len(attribute_data['name'].unique()))

<a id="mfg"></a>
Ta sẽ thử xem tên các thuộc tính xuất hiện với tần suất nào.  
  
Có một trường thông tin là **MFG Brand Name** xuất hiện nhiều nhất, ngữ nghĩa của nó là trường mang thông tin nhà sản xuất sản phẩm đó (**Manufacturing Brand Name**).  

Đây sẽ là một trường thông tin rất hữu ích.  

In [ ]:
attribute_counts = attribute_data.name.value_counts()
print(attribute_counts)

<a id="description_data_exploration"></a>
## Khám phá dữ liệu mô tả.

Hiện ta chưa biết dữ liệu **description** mang thông tin gì.  

Ta sẽ in ra một số dòng đầu của dữ liệu mô tả bằng hàm **pandas.DataFrame.head()**.  
  
Nhìn vào dữ liệu, ta thế dữ liệu này mang 2 thông tin:  
    - **product_uid**: Mã sản phẩm.  
    - **product_description**: Dữ liệu văn bản mô tả sản phẩm. Dữ liệu này bao gồm cả các ký tự latin viết hoa, viết thường, các chữ số và ký tự đặc biệt.    
    
Dữ liệu này không có gì đặc biệt lắm. Tuy nhiên, trường **product_desciption** có thể mang những từ liên quan đến **search_term**, ta sẽ tìm hiểu sau.  

In [ ]:
descriptions.head()

<a id="data_processing"></a>
# Xử lý, phân tích, và xây dựng dữ liệu

<a id="new_brand"></a>
## Xây dựng thông tin brand, và join dữ liệu brand và description

Trước tiên, ta thấy **training_data** và **testing_data** đều mang thông tin giống nhau (chỉ khác là **testing_data** không có trường **relevance**).  

Do đó để xử lý ta sẽ nối chúng với nhau để làm một thể, hơn là chia nhau để xử lý.  

Hàm **pandas.concat** sẽ nối 2 DataFrame với nhau, trường thông tin bị thiếu sẽ được thay bằng **NaN**.  
  
Kết quả thu được là một DataFrame tổng hợp của **training_data** và **testing_data**. Ta có thể nhìn qua một chút về chúng qua hàm **all_data.tail()** (**relevance** của **tesing_data** được ép về NaN), và có tổng cộng 240760 hàng tất cả.

In [ ]:
all_data = pd.concat([training_data, testing_data], axis=0, ignore_index=True)
print(all_data.tail())
len(all_data)

Như đã nhắc đến ở [đây](#mfg), **MFG Brand Name** là một thông tin hữu ích.  
  
Ở đây tôi muốn xây dựng một dataFrame khác chỉ dựa trên thông tin này, bằng việc chọn những hàng ở **attribute_data** có **attribute_data.name == "MFG Brand Name"**, lấy **product_uid** và "**value**", và trường **value** được thay bằng thông tin mới: **brand** - tên nhãn hiệu.  
  
Kết quả thu được là **brand_data**, mang thông tin nhãn hiệu của các sản phẩm, ta có thể nhìn qua chúng một chút qua hàm **brand_data.head()**.  

In [ ]:
brand_data = attribute_data[attribute_data.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
brand_data.head()

Dữ liệu **search_term** có chứa số, và pandas có thể ép kiểu nó thành số ở một số hàng.  
Trong khi rõ ràng ta muốn **search_term** là xâu, ta cần ép kiểu chúng thành **str**.  
  
Ta dùng hàm **astype(str)** để ép chúng thành **str**.  
  
Kết quả thu được là mọi **search_term** đều là xâu, thuận tiện cho sau này.  
  
Ngoài ra, ta cũng làm tương tự với thông tin brand của **brand_data**, sau khi loại bỏ mọi dữ liệu **na**.

In [ ]:
all_data["search_term"]= all_data["search_term"].astype(str)
brand_data.dropna(inplace=True)
brand_data['brand'] = brand_data['brand'].astype(str)

Như đã nói ở [đây](#float-id), ta sẽ muốn ép kiểu **product_uid** của **attribute_data** về dạng **int** để chuẩn hơn trong quá trình chuyển hóa sau này.  
  
Ngoài ra, ta cũng loại bỏ các trường **na** ở **attribute_data** và **descriptions** luôn.  
  
Kết quả là **attribute_data** và **descriptions** đều chỉ mang các dữ liệu chuẩn.

In [ ]:
attribute_data.dropna(inplace=True)
attribute_data['product_uid'] = attribute_data['product_uid'].astype(int)
descriptions.dropna(inplace=True)

Vấn đề ở đây là, **descritions** và **brand_data** đều là những thông tin hữu ích, nhưng nó lại nằm ở các DataFrame khác nhau, khó khăn cho việc sử dụng.  
  
Đều sở hữu trường **product_uid**, ta hoàn toàn có thể join chúng với **all_data**, bằng **left_join**.   
  
Kết quả là **all_data**, xét theo **product_uid**, **all_data** sẽ được "gắn" thêm thông tin **descritions** và **brand_data** tương ứng. Ta có thể dễ dàng xem chúng với hàm **all_data.head()**.    

In [ ]:
all_data = pd.merge(all_data, descriptions, how='left', on='product_uid')
all_data = pd.merge(all_data, brand_data, how='left', on='product_uid')
all_data.head()

<a id="new_attribute"></a>
### Xây dựng attribute_data mới

Hiện ta muốn xây dựng một trường **attribute_data** mới, chỉ gồm 2 trường là **product_uid** và **attribute** (text).  
  
Như đã nói đến ở [đây](#mfg), thông tin "**Bullet**" là vô nghĩa và tôi muốn xóa chúng.  
Trước tiên, tôi tạo **attribute_data_stripped** mới (để tránh thay đổi dữ liệu gốc), với hàm **re.sub** để xóa đi các "**Bullet**", thay bằng xâu rỗng.

In [ ]:
attribute_data_stripped = attribute_data
attribute_data_stripped['name'] = attribute_data_stripped['name'].apply(lambda s: re.sub(r"Bullet([0-9]+)", "", s))

Sau bước bên trên, giờ attribute sẽ cần mang dữ liệu từ cả **name** và **value**, do đó ta cộng xâu lại.  
  
Ta xây dựng trường **attribute** bằng việc cộng xâu, với khoảng trắng ở giữa, giữa tên **attribute** và trường **value**.  
  
Sau bước này, dữ liệu **attribute** đã có và sẵn sàng để tại DataFrame mới.  

In [ ]:
attribute_data_stripped['attribute'] = attribute_data_stripped['name'] + " " + attribute_data_stripped['value']

Sau bước trên, ta vẫn thấy có vấn đề, đó là một **product_uid** chứa nhiều **attribute**.  
  
Do đó, ta nhóm hết các dữ liệu bằng hàm **groupby('product_uid')**, sau đó cộng xâu chúng lại bằng hàm **' '.join(...)** với **agg()**, phân tách bởi dấu cách.  
  
Kết quả là **attribute_data_new**, với 2 trường là **product_uid** và **attribute**, sẵn sàng cho việc gộp vào dữ liệu chính **all_data** như **brand_data** và **descriptions**.

In [ ]:
attribute_data_new = attribute_data_stripped.groupby('product_uid').agg({'attribute': lambda s : ' '.join(s.astype(str))}).reset_index()
attribute_data_new.head()

<a id="normalize"></a>
**attribute_data** vẫn đứng độc lập, giờ ta cần gộp nó vào với **all_data**.  
  
Giống như với brand_data và descriptions, ta sẽ thực hiện **left_join** giữa **all_data** và **attribue_data_new**, dựa trên **product_uid** bằng hàm **pandas.merge()**.  
  
Kết quả thu được là **all_data** với thêm thông tin **attribute**.

In [ ]:
all_data = pd.merge(all_data, attribute_data_new, how = 'left', on = 'product_uid')
all_data.head()

<a id="pre_processing"></a>
## Tiền xử lý

Nhắc đến việc tiền xử lý, ta nghĩ đến một số bước phổ biến sau:  
* Chuẩn hóa xâu (viết thường hết, loại bỏ các ký tự đặc biệt, ....)  
* Thay những từ cùng nghĩa nhưng có cách viết khác (ví dụ như số nhiều, số ít, chia thì, ...)  
  
Trước tiên ta sẽ Import những thư viện cần thiết, chuẩn bị các hàm cho việc xử lý.  

In [ ]:
from nltk.stem.snowball import SnowballStemmer
import re

<a id="normalization"></a>
### Chuẩn hóa xâu

Như đã nhắc đến bên trên, đầu tiên là việc chuẩn hóa xâu.  
  
Ta xây dựng hàng **normalize_string** với input là 1 xâu, kết quả trả về một xâu đã chuẩn hóa.  
Các bước chuẩn hóa được chú thích từng dòng trong code.  
Để việc xử lý thuận tiện, ta sử dụng ký pháp **regex** và thư viện **re** của python.  
Việc này giúp tốc độ xử lý xâu đạt cao nhất.  
  
Sau cell này ta đã có hàm chuẩn hóa xâu **normalize_string**, sau này nếu muốn dùng ta chỉ việc **apply** nó cho các thuộc tính của DataFrame.

In [ ]:
def normalize_string(input_str: str):
    # Loại bỏ tất cả các ký tự đặc biệt
    input_str = re.sub('\W', ' ', input_str.lower())
    # Loại bỏ các ký tự đứng một mình
    input_str = re.sub('\s+[a-zA-Z]\s+', ' ', input_str)
    # Sau bước trên, còn sót lại các ký tự đứng 1 mình ở đầu câu
    # Ta xóa nốt trường hợp này
    input_str = re.sub(r'\^[a-zA-Z]\s+', ' ', input_str) 
    # Sẽ có nhiều dấu cách đứng cạnh nhau sau bước trên 
    # Ta sẽ thay toàn bộ các dấu cách đứng cạnh nhau thành 1 dấu duy nhất 
    input_str = re.sub('\s+', ' ', input_str, flags=re.I)
    
    # Các số cũng rất quan trọng, tuy nhiên ta không muốn các số xuất hiện trong xâu.
    # Ta thay các chữ số thành các chữ tiếng Anh tương ứng
    input_str = re.sub(r"zero\.?", r"0 ", input_str)
    input_str = re.sub(r"one\.?", r"1 ", input_str)
    input_str = re.sub(r"two\.?", r"2 ", input_str)
    input_str = re.sub(r"three\.?", r"3 ", input_str)
    input_str = re.sub(r"four\.?", r"4 ", input_str)
    input_str = re.sub(r"five\.?", r"5 ", input_str)
    input_str = re.sub(r"six\.?", r"6 ", input_str)
    input_str = re.sub(r"seven\.?", r"7 ", input_str)
    input_str = re.sub(r"eight\.?", r"8 ", input_str)
    input_str = re.sub(r"nine\.?", r"9 ", input_str)

    return input_str

Từ [đây](#normalize), ta sẽ chuẩn hóa mọi thông tin là xâu, bao gồm:  
* **search_term**  
* **product_title**  
* **product_description**  
* **attribute**  
điều này có thể được thực hiện bằng hàm **apply(normalize_string)**.  
  
Do thời gian chạy khá lâu nên ta import thêm thư viện time để tính xem đã mất bao nhiều thời gian.  
  
Kết quả thu được là **all_data** đã được chuẩn hóa mọi xâu.

In [ ]:
import time

start_time = time.time()

all_data['search_term'] = all_data['search_term'].apply(str).apply(normalize_string)
all_data['product_title'] = all_data['product_title'].apply(str).apply(normalize_string)
all_data['product_description'] = all_data['product_description'].apply(str).apply(normalize_string)
all_data['attribute'] = all_data['attribute'].apply(str).apply(normalize_string)
print(all_data.head())

print("--- %s seconds ---" % (time.time() - start_time))

<a id="stem"></a>
### Stemming

Như đã nhắc đến, ta muốn thay những từ gần giống nhau chỉ bằng 1 từ duy nhất (ví dụ như số nhiều, số ít, chia thì, ...), đây gọi là quá trình "**stemmed**".  
  
Ta sẽ sử dụng thư viện đã hỗ trợ của **nltk**, đó là **SnowballStemmer** với ngôn ngữ là english.  
Với mỗi từ có được bằng hàm **str.split()**, ta thay nó bằng từ gốc (nếu có).  
  
Kết quả ta được hàm **str_stemmer**, nhận đầu vào là một xâu, và trả về một xâu đã được "**stemmed**".  

In [ ]:
stemmer = SnowballStemmer('english')

def str_stemmer(s):
    return " ".join([stemmer.stem(word) for word in s.lower().split()])

Giờ ta áp dụng hàm trên vào **all_data** với hàm **apply(str_stemmer)**.  
  
Tương tự như phần chuẩn hóa xâu, ta cũng đo thời gian chạy.  
  
Kết quả thu được là **all_data** với các xâu đã chỉ chứa những từ gốc, chứ không còn biến thể nữa.

In [ ]:
start_time = time.time()

all_data['search_term'] = all_data['search_term'].apply(str).apply(str_stemmer)
all_data['product_title'] = all_data['product_title'].apply(str).apply(str_stemmer)
all_data['product_description'] = all_data['product_description'].apply(str).apply(str_stemmer)
all_data['attribute'] = all_data['attribute'].apply(str).apply(str_stemmer)

print("--- %s seconds ---" % (time.time() - start_time))

<a id="new_quantitative"></a>
## Tạo ra một số dữ liệu định lượng

Để có thể xây dựng thêm các trường thông tin, ta cần biết nó có ảnh hưởng thế nào đến **relevance** - yếu tố ta cần đoán.  
  
Để ý đến độ dài của các trường thông tin văn bản, ta sẽ vẽ thử chúng trên đồ thị và 1 đường **regression**.  
Ta sử dụng thư viện **matplotlib** để vẽ đồ thị này.  
Chi tiết hơn được giải thích trong code.
  
Ta được một hàm see_correlation vẽ ra sự tương quan giữa độ dài của một trường thông tin bất kỳ với **relevance**. Tham số **transform** sẽ bằng **True** nếu ta muốn lấy độ dài của **data_field**, và bằng **False** nếu ta muốn lấy luôn giá trị của **data_field**.

In [ ]:
import matplotlib.pyplot as plt

def see_correlation(data_field, transform=True):
    # Ta chỉ lấy 1 phần từ dữ liệu ban đầu để cho hiệu quả tốt nhất
    # Và ta chỉ lấy phần của training_data, thứ chứa relevance
    data_sample = all_data[:len(training_data) - 1].sample(frac=.1)
    # Lấy độ dài của trường thông tin
    x_ar = np.array(data_sample[data_field].map(lambda x:len(str(x).split())).astype(np.int64)) if transform else data_sample[data_field]
    # Lấy relevance tương ứng
    y_ar = np.array(data_sample['relevance'])
    # Vẽ các điểm dữ liệu lên đồ thị
    plt.plot(x_ar, y_ar, 'o')
    # Tìm tham số m và b để vẽ đường regression
    m, b = np.polyfit(x_ar, y_ar, 1)
    # Vẽ được regression
    plt.plot(x_ar, m * x_ar + b)

Như đã giải thích ở trên, giờ ta sẽ vẽ thử ra sự tương quan của **relevance** với lần lượt độ dài của từng trường thông tin, bắt đầu với **product_title**.  
  
Theo kết quả, các điểm **relevance** phân bố rất đều nhau trên mọi độ dài **product_title**, đường regression gần như đi ngang. Ta có thể nhận xét rằng **product_title** gần như không ảnh hưởng gì đến **relevance** cả.  
  
Do đó, do sự không khả quan, sau này ta sẽ không sử dụng đặc trưng này vào khâu huấn luyện.

In [ ]:
see_correlation('product_title')

**Product_title** không mang lại kết quả, ta tiếp tục với **search_term**.  
  
Theo kết quả, các điểm relevance phân bố khá đều trên mọi độ dài **search_term**, tuy nhiên lại có nhiều điểm khác thường mà theo xu hướng nào đó. Đường regression có độ dốc khá lớn.  
  
Ta có thể nhận xét rằng **search_term** có ảnh hưởng đến **relevance**.  
  
Do đó, với kết quả khả quan khả quan, ta sẽ sử dụng đặc trưng này vào khâu huấn luyện.  

In [ ]:
see_correlation('search_term')

Ta tiếp tục với **product_description**.  
  
Theo kết quả, các điểm relevance phân bố khá đều trên mọi độ dài **product_description**, tuy nhiên lại có nhiều điểm khác thường mà theo xu hướng nào đó.  
Đường regression có độ dốc đáng cũng khá đáng kể.  
  
Ta có thể nhận xét rằng **product_description** có ảnh hưởng đến **relevance**.  
  
Do đó, cũng với kết quả khả quan khả quan, ta sẽ sử dụng đặc trưng này vào khâu huấn luyện. 

In [ ]:
see_correlation('product_description')

Ta tiếp tục với **brand**.  
  
Theo kết quả, các điểm **relevance** phân bố khá đều trên mọi độ dài **brand**, tuy nhiên lại có nhiều điểm khác thường mà theo xu hướng nào đó.  
Đường regression có độ dốc đáng cũng khá đáng kể.  
  
Ta có thể nhận xét rằng brand có ảnh hưởng đến **relevance**.  
  
Do đó, cũng với kết quả khả quan khả quan, ta sẽ sử dụng đặc trưng này vào khâu huấn luyện. 

In [ ]:
see_correlation('brand')

Ta tiếp tục với **attribute**.  
  
Theo kết quả, các điểm relevance phân bố khá đều trên mọi độ dài **attribute**, tuy nhiên lại có nhiều điểm khác thường mà theo xu hướng nào đó.  
Đường regression có độ dốc đáng cũng khá đáng kể.  
  
Ta có thể nhận xét rằng **attribute** có ảnh hưởng đến **relevance**.  
  
Do đó, cũng với kết quả khả quan khả quan, ta sẽ sử dụng đặc trưng này vào khâu huấn luyện. 

In [ ]:
see_correlation('attribute')

Dựa vào các đồ thị regression trên, ta thấy chiều dài của các thông tin, trừ **title** và **attribute** cũng có ảnh hưởng đến relevance.  
  
Do đó ta đưa thông tin chiều dài title bằng hàm **len(title)** vào thành một trường thông tin khác.  
Độ dài của **query**, **brand**, **descrition**, **attribute** cũng tương tự.  
  
Ta thêm vào **all_data** các thông tin sau, là độ dài các thông tin nội sinh của dữ liệu, bao gồm:
* **len_of_query**: Số từ (ngăn cách bởi dấu cách) trong cụm từ tìm kiếm  
* **len_of_brand**: Độ dài số từ trong tên nhãn hiệu  
* **len_of_description**: Độ dài đoạn văn bản mô tả của sản phẩm  

In [ ]:
all_data['len_of_query'] = all_data['search_term'].map(lambda x:len(str(x).split())).astype(np.int64)

all_data['len_of_brand'] = all_data['brand'].map(lambda x:len(str(x).split())).astype(np.int64)

all_data['len_of_description'] = all_data['product_description'].map(lambda x:len(str(x).split())).astype(np.int64)

Dựa vào chính thực tế sử dụng các công cụ tìm kiếm, ta đoán rằng các search engine thường dựa vào các từ trong **search_term** mà xuất hiện trong văn bản.  
  
Do đó, ta sẽ đếm số lượng các từ nằm trong từ khóa tìm kiếm (**search_term**) mà xuất hiện trong:  
* Tiêu đề: **word_in_title**  
* Mô tả sản phẩm: **word_in_description**  
  
Để làm được điều này, ta định nghĩa hàm **str_common_word**, nhận vào 2 xâu, trả về số lượng từ trong xâu 1 đã xuất hiện trong xâu 2.  

In [ ]:
def str_common_word(str1, str2):
    return sum([1 if str2.find(word) >= 0 else 0 for word in set(str1.split())])

Trước tiên ta tạo một trường thông tin là **product_info** để tiện xử lý sau này.  
Thông tin này được gộp từ tất cả các dữ liệu mang thông tin sản phẩm:  
* **search_term**: Cụm từ tìm kiếm.  
* **product_title**: Tiêu đề sản phẩm  
* **product_description**: Mô tả sản phẩm  
  
Ta gộp chúng lại, phân tách bằng ký tự "\t", để sau này tiện tách ra.  
  
Kết quả thu được là **all_data** với trường **product_info**.

In [ ]:
all_data['product_info'] = all_data['search_term'] + '\t' + all_data['product_title'] + '\t' + all_data['product_description']

Giờ ta cần tính **word_in_title** và **word_in_description** mà đã nhắc đến bên trên.  
  
Từ **product_info**, đầu tiên ta tách 3 thông tin ra bằng hàm split('\t'), thì kết quả đầu tiên là **search_term**, thứ 2 là **title**, thứ 3 là **description**.  
Tiếp đó, ta sử dụng hàm **str_common_word** đã định nghĩa để đếm số từ trong **search_term** trong lần lượt 2 thông tin **title** và **description**.  
  
Kết quả là 2 trường thông tin mới trong **all_data**, là **word_in_title** và **word_in_description**.

In [ ]:
all_data['word_in_title'] = all_data['product_info'].map(lambda x:str_common_word(str(x).split('\t')[0], str(x).split('\t')[1]))
all_data['word_in_description'] = all_data['product_info'].map(lambda x:str_common_word(str(x).split('\t')[0], str(x).split('\t')[2]))

Giờ ta muốn đánh giá xem các thông tin mới này có ảnh hưởng đến **relevance** không.
  
Ta dùng lại hàm **see_correlation**, với tham số thứ 2 là **False**, tức là không cần lấy **len** - độ dài văn bản nữa mà lấy trực tiếp dữ liệu luôn.  
  
Bắt đầu với **word_in_description**, với đường regression có độ dốc rất lớn, ta thấy nó ảnh hưởng khá nhiều đến **relevance**.  
  
Đây sẽ là một thông tin hữu ích mà ta sử dụng cho việc huấn luyện.

In [ ]:
see_correlation('word_in_description', False)

Tiếp tục với **word_in_title**, làm tương tự bên trên.  
  
Ta thấy nó ảnh hưởng khá lớn đến relevance khi đường regression có độ dốc khá lớn, đây cũng sẽ là một thông tin hữu ích.

In [ ]:
see_correlation('word_in_title', False)

Ta thấy một vấn đề, đó là số lượng có thể không phải là một con số cụ thể về sự tương quan giữa các dữ liệu.  
Ta còn cần phải xét đến tỷ lệ xuất hiện trong các ngữ cảnh nữa.  
Cụ thể, dựa vào số đếm vừa tính bên trên, ta tính tỷ lệ các từ xuất hiện trong **search_term** của các trường thông tin sau:  
* Tiêu đề: **ratio_title**  
* Mô tả: **ratio_description**  
Việc tính 2 chỉ số này cũng đơn giản, ta chỉ cần lấy thương của 2 trường thông tin đã có, **word_in_title** / **word_in_description** tương ứng, với mẫu là **all_of_query**.
  
Kết quả là **all_data** với thêm 2 trường, là **ratio_title** và **ratio_description**.

In [ ]:
all_data['ratio_title'] = all_data['word_in_title'] / all_data['len_of_query']
all_data['ratio_description'] = all_data['word_in_description'] / all_data['len_of_query']

Giờ ta muốn biết chúng có ảnh hưởng đến **relevance** hay không. Ta bắt đầu với **ratio_title**.  
  
Ta tiếp tục sử dụng hàm **see_correlation** như trên.  
  
Đường regression chạy từ **relevance** bằng **2.0** đến gần **2.75**, kết quả cho thấy đây là một thông tin rất hữu ích, và ta có thể sử dụng thông tin này như một feature cho việc huấn luyện.

In [ ]:
see_correlation('ratio_title', False)

Giờ ta muốn biết **ratio_description** có ảnh hưởng đến **relevance** hay không.  
  
Ta tiếp tục sử dụng hàm **see_correlation** như trên.  
  
Đường regression chạy từ **relevance** bằng **2.0** đến hơn **2.5**, kết quả cho thấy đây là một thông tin rất hữu ích, và ta có thể sử dụng thông tin này như một feature cho việc huấn luyện.

In [ ]:
see_correlation('ratio_description', False)

Hiện tại, còn duy nhất một thông tin mà ta chưa nhắc đến, đó là bản thân văn bản của **description**, **brand**, và **attribute**.  
  
Để sử dụng cho bước sau, ta gộp tiếp 3 trường thông tin: **descriptions**, **brand**, **attribute** lại thành một.  
  
Kết quả là **all_data** có thêm trường **prod_desc_merge**.

In [ ]:
all_data["prod_desc_merge"] = all_data["product_description"].map(str) +' ' + all_data["brand"].fillna('') + ' ' + all_data["attribute"].fillna('').map(str)

Ta nhận thấy trong quá trình, có rất nhiều cột đã được sinh mới ra nhưng không dùng đến, hoặc chúng đã được gộp lại để tiện xử lý.  
  
Ta tạo một cells để xóa chúng đi, sử dụng làm **pandas.DataFrame.drop(inplace=True, axis=1)**, tức là xóa theo cột, và xóa luôn trong **all_data**.  
  
Kết quả thu được là **all_data** chỉ còn các trường thông tin mà ta cho là hữu ích.

In [ ]:
for column in ["attr", "product_description", "brand", "attribute", "product_attributes", "product_info", 'last_word_title_match', 'first_word_title_match', 'last_word_description_match', 'first_word_description_match']:
    if column in all_data:
        if column in all_data:
            all_data.drop(column, axis=1, inplace=True)

<a id="data_representation"></a>
# Biểu diễn dữ liệu

<a id="tfidf"></a>
## TF-IDF

Hiện ta cần biểu diễn các dữ liệu xâu cuối cùng thành số, bao gồm **search_term**, **product_title**, và **prod_desc_merge**.  
  
Ở bước này, ta sử dụng **tf-idf**.  
  
**TF-iDF** là một phép thống kê, với tác dụng phản ánh tầm quan trọng của một từ / dấu hiệu (token) với tập hợp văn bản. Giá trị của **tf-idf** tăng tỷ lệ thuận với số lần xuất hiện của từ đó trong tài liệu, và được giảm đi với số lượng các văn bản chứa từ / dấu hiệu này. Mô hình này cũng được cung cấp trong thư viện scikit-learn.  
  
Kết quả thu được sau cell này là một object **tfidf** với **ngram_range** là 1 đến 2, tức là xét các từ đơn và từ ghép 2 từ, cùng với đó là loại bỏ các **stop_words** trong tiếng Anh.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')

<a id="svd"></a>
## TruncatedSVD

Số chiều của các dữ liệu cũng rất lớn.  
  
Để xử lý việc này, làm cho tốc độ xử lý được nhanh hơn, ta sử dụng **TruncatedSVD** để giảm chiều của dữ liệu.  
  
Kết quả sau cell này là một object của **truncatedSVD** với số chiều là 100 - con số tôi thử nghiệm mà đem lại kết quả khả quan trong thời gian chấp nhận được.

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, random_state = 2021)

<a id="tfidf_svd"></a>
## Pipeline của dữ liệu

Giờ ta muốn dữ liệu sẽ được xử lý lần lượt bằng **tfidf** và **svd**, tuy nhiên việc viết fit 2 lần sẽ làm ta code bất tiện hơn.  
  
Do đó ta sử dụng **PipeLine** từ thư viện **sklearn**, cho ta nối **tfidf** với **svd**.  
  
Kết quả của cell là một pipeline, cho phép dữ liệu đầu vào đi qua mọi khâu trong **pipeline**, rất thuận tiện cho việc xử lý.

In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[('tfidf', tfidf), ('svd', svd)])

Ở bước này, như đã nói, ta xử lý:
* **product_title**  
* **search_term**  
* **prod_desc_merge**  
với pipeline ở trên, thông qua hàm **pipe.fit_transform(input_data)**.  
  
Kết quả là các trường thông tin trên được biểu diễn dưới dạng máy có thể dễ dàng hiểu được, chuẩn bị cho khâu huấn luyện.  
  
Ngoài ra, khâu này cũng sẽ mất nhiều thời gian nên tôi muốn đo thời gian chạy bằng thư viện time.

In [ ]:
start_time = time.time()

all_data["product_title"] = pipe.fit_transform(all_data["product_title"])
all_data["search_term"] = pipe.fit_transform(all_data["search_term"])
all_data["prod_desc_merge"] = pipe.fit_transform(all_data["prod_desc_merge"])

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# import pickle

# with open('processed_transformed.data', 'wb') as picklefile:
#     pickle.dump(all_data, picklefile)

In [ ]:
# import pickle

# with open('../input/processed-data-sets/processed_transformed.data', 'rb') as object_to_be_loaded:
#     all_data = pickle.load(object_to_be_loaded)
    
# print(len(all_data))
# all_data.head()

<a id="loss"></a>
## Hàm mất mát

Giờ ta cần một hàm đánh giá sai số của mô hình.  
  
Như đề bài đã mô tả cách đánh giá kết quả, ta cũng sử dụng hàm **mean_squared_error**, đã được cung cấp trong thư viện scikit-learn.  
Ta cũng sử dụng hàm **make_scorer** trong thư viện này để tạo ra một đối tượng để thư viện huấn luyện gọi được, với tham số **greater_is_better=False**, tức là hàm càng trả về kết quả nhỏ thì càng tốt.  
  
Kết quả của cell này là định nghĩa hàm **fmean_square_error** và đối tượng **RMSE** để sau này sử dụng cho việc đánh giá sai số của mô hình.

In [ ]:
from sklearn.metrics import mean_squared_error, make_scorer

def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.1
    return fmean_squared_error_

RMSE = make_scorer(fmean_squared_error, greater_is_better=False)

<a id="data_split"></a>
## Phân tách dữ liệu train-test

Hiện giờ mọi dữ liệu đều đã được xử lý, tuy nhiên chúng ta lại cần tách ra dưới dạng **training_data** và **testing_data**.  
  
Ta dùng hàm **all_data.iloc** để tách lại dữ liệu **train** và **test**.  
Bởi thứ tự của các mẫu không thay đổi, ta chỉ đơn giản là lấy theo số lượng mẫu trong **training_data**.  
  
Kết quả sau cell này là 2 dataFrames, **df_train** - dữ liệu dùng để train, và **df_test** - dữ liệu test để lấy kết quả nộp bài.

In [ ]:
df_train = all_data.iloc[:len(training_data)]
df_test = all_data.iloc[len(training_data):]

Giờ ta cần tách rõ ràng **X_train**, **X_test**, **y_train** để huấn luyện.  
  
Ta sẽ lấy các thông tin này như sau:  
* X_train: là **df_train**, nhưng không có trường **relevance**  
* y_train: là **df_train** chỉ với trường **relevance**  
* X_test: là **df_test**, nhưng không có trường **relevance**  
  
Ngoài ra, ta còn tạo thêm một dữ liệu khác, là **id_test**, để thực hiện việc lấy kết quả dự doán và submit sau này.  
  
Kết quả sau cell này là các dữ liệu **X_train**, **X_test**, **y_train**, và **id_test** như đã mô tả.

In [ ]:
# là df_train, nhưng không có trường relevance 
X_train = df_train.drop('relevance', axis=1)
# là df_train chỉ với trường relevance
y_train = df_train['relevance'].values
# là df_test, nhưng không có trường relevance
X_test = df_test.drop('relevance', axis=1)

# id_test để thực hiện dự đoán và submit kết quả sau này
id_test = df_test['id']

# Huấn luyện mô hình

<a id="model_selection"></a>
## Khởi tạo mô hình 

Giờ ta chỉ cần định nghĩa, thử nghiệm các mô hình.  
  
Đầu tiên, tôi muốn thử **GradientBoostingRegressor**, một mô hình ensemble khá nổi tiếng, đã được cung cấp trong thư viện scikit-learn.  
  
Kết quả là một object của mô hình **GBR** này.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()

<a id="find_best_param"></a>
## Tìm tham số tối ưu

Mô hình để chạy tốt được cần phải có bộ tham số tối ưu.  
  
Qua tham khảo, tôi thấy có 2 trường thông tin quan trọng có thể hiệu chỉnh, đó là **n_estimators** và **max_depth**:
* **n_estimators**: Số lượng khâu boost cần thực hiện. Vì tốc độ, và qua tham khảo, tôi muốn lấy n_estimators là các số nguyên trong khoảng 30-40, với hàm **numpy.linspace**.     
* **max_depth**: Độ sâu giới hạn của các node. Theo tham khảo, tôi thử nghiệm trên các số {2, 4, 6, 8, 10}, với hàm **numy.linspace**.     
  
Kết quả của cell này là một dictionary mang các tham số cần hiệu chỉnh cho cell sau.

In [ ]:
param_grid = {
    'n_estimators' : np.array([int(e) for e in np.linspace(30, 40, 11)]),
    'max_depth': np.array([int(e) for e in np.linspace(2, 10, 5)])
}

Giờ ta cần chạy mô hình nhiều lần để tìm được bộ tham số tối ưu.  
  
Thư viện scikit-learn đã cung cấp class **GridSearchCV** cho việc này. Ta sẽ truyền vào đối tượng cần tối ưu là **gbr** đã định nghĩa bên trên, **param_gid=param_grid** đã định nghĩa, là bộ các tham số cần tìm tối ưu, hàm điểm **scorring=RMSE** đã định nghĩa trước đó, với **cv=3**, là 3 lần kiểm định chéo **cross-validation**.  
  
Kết quả cell này là đối tượng gs để tìm bộ tham số tối ưu.

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=3, scoring=RMSE)

Giờ công việc của ta là chạy object **gs** với **X_train**, đối chiếu trên **y_train**, để tìm ra bộ tham số tối ưu.  
  
Ta sử dụng hàm **gs.fit(X_train, y_train)**.  
  
Kết quả của cell này là object **gs** đã chứa những tham số, và điểm tối ưu có được trong các khâu cross-validation.  

In [ ]:
%time _ = gs.fit(X_train, y_train)

Như đã nói ở cell trên, giờ ta chỉ việc in ra bộ tham số tối ưu.  
  
Ta sẽ gọi **gs.best_params_** để in ra bộ tham số tối ưu, và **gs.best_score_** để in ra điểm tối ưu đã đạt được.  
  
Kết quả của cell này sẽ được lấy để huấn luyện mô hình cuối và dự đoán kết quả cuối cùng, nộp lên kaggle để tính điểm.

In [ ]:
gs.best_params_, gs.best_score_

<a id="best_param_training"></a>
## Huấn luyện mô hình với tham số tối ưu

Giờ ta cần định nghĩa mô hình cuối cùng, dựa trên bộ tham số tối ưu đã có.  
  
Ta khởi tạo lại **gbr_best** bằng **GradientBoostingRegressor**, với bộ tham số tối ưu có bên trên.  
Sau cùng, ta gọi hàm **gbr_best.fit(X_train, y_train)** để huấn luyện mô hình.  
  
Kết quả là mô hình GradientBoostingRegressor tối ưu mà ta sẽ sử dụng để dự đoán dữ liệu test, còn nộp lên lấy kết quả.

In [ ]:
gbr_best = GradientBoostingRegressor(max_depth=gs.best_params_['max_depth'],
                                     n_estimators=gs.best_params_['n_estimators'])

gbr_best.fit(X_train, y_train)

<a id="prediction"></a>
## Dự đoán kết quả và nộp

Giờ công việc của là cần dự đoán kết quả của **X_test**.  
  
Ta gọi **gbr_best.predit(X_test)** để được kết quả dự đoán.  
  
Kết quả là **y_pred** mang các giá trị **relevance** mà ta dự đoán được.

In [ ]:
y_pred = gbr_best.predict(X_test)

Cuối cùng, ta cần lưu lại file output theo đúng format để nộp.  
  
Ta sử dụng hàm **pandas.DataFrame.to_csv** để lưu lại file, với 2 trường thông tin là **id** và **relevance** - **y_pred** đã tính được.  

In [ ]:
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_gbr.csv', index=False)

<a id="result"></a>
# Kết quả  
  
Private score là **0.48009**